In [1]:
import os
import glob
import torch
import warnings
import numpy as np
from pathlib import Path
from sklearn.utils import shuffle
import util
import torch
import torchaudio
import torchaudio.transforms as T
import torchvision.transforms as transforms
from pathlib import Path
#from feature_extractor import FeatureExtractor
#from utils import prepare_input_features
#from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
warnings.filterwarnings(action='ignore')

In [2]:
def cut_if_necessary(signal, num_samples):
    """return signal"""
    if signal.shape[1] > num_samples:
        signal = signal[:, :num_samples]
    return signal

In [3]:
def right_pad_if_necessary(signal, num_samples):
    """return signal"""
    length_signal = signal.shape[1]
    if length_signal < num_samples:
        num_missing_samples = num_samples - length_signal
        last_dim_padding = (0, num_missing_samples)
        signal = torch.nn.functional.pad(signal, last_dim_padding)
    return signal

In [4]:
train_speaker_clean = util.load_variables('variables/train_speaker_identification_clean.pkl')
val_speaker_clean = util.load_variables('variables/val_speaker_identification_clean.pkl')
test_speaker_clean = util.load_variables('variables/test_speaker_identification_clean.pkl')

In [5]:
train_speaker_noises_clean = util.load_variables('variables/train_speaker_noises_identification_noise_clean.pkl')
val_speaker_noises_clean = util.load_variables('variables/val_speaker_noises_identification_noise_clean.pkl')
test_speaker_noises_clean = util.load_variables('variables/test_speaker_noises_identification_noise_clean.pkl')

In [6]:
train_speaker_denoise = util.load_variables('variables/train_speaker_identification_clean_dnn.pkl')
val_speaker_denoise = util.load_variables('variables/val_speaker_identification_clean_dnn.pkl')
test_speaker_denoise = util.load_variables('variables/test_speaker_identification_clean_dnn.pkl')

In [7]:
train_speaker_noises_dnn = util.load_variables('variables/train_speaker_noises_identification_noise_dnn.pkl')
val_speaker_noises_dnn = util.load_variables('variables/val_speaker_noises_identification_noise_dnn.pkl')
test_speaker_noises_dnn = util.load_variables('variables/test_speaker_noises_identification_noise_dnn.pkl')

In [8]:
def generate_only_list_speaker(list_speaker1, list_speaker2):
    only_list_speakers = []
    for idx, (speaker1, speaker2) in enumerate(zip(list_speaker1, list_speaker2)):
        #print(speaker1)
        #print(speaker2)
        #print(speaker3)
        only_list_speakers.append([speaker1, speaker2])
    return only_list_speakers

In [9]:
only_list_speakers_train_clean_noise = generate_only_list_speaker(train_speaker_clean, train_speaker_noises_clean)
only_list_speakers_val_clean_noise = generate_only_list_speaker(val_speaker_clean, val_speaker_noises_clean)
only_list_speakers_test_clean_noise = generate_only_list_speaker(test_speaker_clean, test_speaker_noises_clean)

In [10]:
only_list_speakers_train_dnn_noise = generate_only_list_speaker(train_speaker_denoise, train_speaker_noises_dnn)
only_list_speakers_val_dnn_noise = generate_only_list_speaker(val_speaker_denoise, val_speaker_noises_dnn)
only_list_speakers_test_dnn_noise = generate_only_list_speaker(test_speaker_denoise, test_speaker_noises_dnn)

In [11]:
#DEFINE PARAMETER TO EXTRACT MFCC FOR EACH AUDIO
# MFCC PARAMETER
SAMPLE_RATE_T = 16000
N_MFCC_T = 40
N_FFT_T = 256
HOP_LENGTH_T = round(0.25*N_FFT_T)
N_MELS_T = N_FFT_T//2+1
CENTER_T = True

In [12]:
# WE DEFINE TO TRANSFORM AUDIO AND SO GET MFCC FROM AUDIO
mfcc_speakers = T.MFCC(
    sample_rate = SAMPLE_RATE_T,
    n_mfcc = N_MFCC_T,
    melkwargs = {
        "n_fft": N_FFT_T,
        "hop_length": HOP_LENGTH_T,
        "n_mels": N_MFCC_T,
        "center": CENTER_T
    }
)
# NORMALISATION FROM MFCC
norm = torch.nn.InstanceNorm1d(N_MFCC_T, affine=False)

In [13]:
def generete_mfcc_pt(list_speakers, mfcc_tranforms, path_mfcc_folder):
    n_frame = 512
    count = 0
    num_samples_time = 16000*5
    LIST_MFCC_OF_ALL_SPEAKERS = []
    for speaker in list_speakers:
        MFCC_CLEAN_SPEAKER_AUX = []
        MFCC_NOISE_SPEAKER_AUX = []
        MFCC_DENOISE_SPEAKER_AUX = []
        for element in speaker:
            #print(element)
            
            path_speaker_element = element[1].split('/')[1]
            print(path_speaker_element)
            
            #print(path_speaker_element)
            if(path_speaker_element=='Speakers'):
                print('Speakers')
                path_mfcc = element[1].split('/')
                print(os.path.join(path_mfcc_folder[0], "/".join(path_mfcc[2:len(path_mfcc)-1])))
                Path(os.path.join(path_mfcc_folder[0], "/".join(path_mfcc[2:len(path_mfcc)-1]))).mkdir(parents=True, exist_ok=True)
                print(element[1])
                audio, sample_rate = torchaudio.load(element[1])
                audio = cut_if_necessary(audio, num_samples_time)
                audio = right_pad_if_necessary(audio, num_samples_time)
                mfcc = mfcc_tranforms(audio).squeeze(0)
                print(mfcc.size())
                for j in np.arange(mfcc.shape[1] // n_frame):
                    aux = mfcc.transpose(0,1)[j * n_frame:(j + 1) * n_frame]
                    out_name =  str(element[0]) +'_'+ str(j).zfill(6) +'_'+ element[1].split('/')[4].split('.')[0] + '.pt'
                    print(out_name)
                    torch.save(aux, os.path.join(path_mfcc_folder[0], "/".join(path_mfcc[2:len(path_mfcc)-1]))+'/'+out_name)
                    MFCC_CLEAN_SPEAKER_AUX.append(os.path.join(path_mfcc_folder[0], "/".join(path_mfcc[2:len(path_mfcc)-1]))+'/'+out_name)
                    count += 1
            
            if(path_speaker_element=='Speakers_with_noise_clean'):
                print('Speakers_with_noise')
                path_mfcc = element[1].split('/')
                print(os.path.join(path_mfcc_folder[1], "/".join(path_mfcc[2:len(path_mfcc)-1])))
                Path(os.path.join(path_mfcc_folder[1], "/".join(path_mfcc[2:len(path_mfcc)-1]))).mkdir(parents=True, exist_ok=True)
                print(element[1])
                audio, sample_rate = torchaudio.load(element[1])
                audio = cut_if_necessary(audio, num_samples_time)
                audio = right_pad_if_necessary(audio, num_samples_time)
                mfcc = mfcc_tranforms(audio).squeeze(0)
                print(mfcc.size())
                for j in np.arange(mfcc.shape[1] // n_frame):
                    aux = mfcc.transpose(0,1)[j * n_frame:(j + 1) * n_frame]
                    out_name =  str(element[0]) +'_'+ str(j).zfill(6) +'_'+ element[1].split('/')[4].split('.')[0] + '.pt'
                    print(out_name)
                    torch.save(aux, os.path.join(path_mfcc_folder[1], "/".join(path_mfcc[2:len(path_mfcc)-1]))+'/'+out_name)
                    MFCC_NOISE_SPEAKER_AUX.append(os.path.join(path_mfcc_folder[1], "/".join(path_mfcc[2:len(path_mfcc)-1]))+'/'+out_name)
                    count += 1
            
        LIST_MFCC_OF_ALL_SPEAKERS.append([MFCC_CLEAN_SPEAKER_AUX, MFCC_NOISE_SPEAKER_AUX])
    print(count)
    return LIST_MFCC_OF_ALL_SPEAKERS

In [14]:
# only_list_speakers_train_clean_noise
# only_list_speakers_val_clean_noise
# only_list_speakers_test_clean_noise

In [15]:
path_mfcc_folder_train_c_n= ['data/MFCC_Speaker_Clean_N_Train/', 'data/MFCC_Speaker_Noise_N_Train/']
LIST_MFCC_OF_ALL_SPEAKERS_TRAIN_C_N = generete_mfcc_pt(only_list_speakers_train_clean_noise, mfcc_speakers, path_mfcc_folder_train_c_n)

Speakers
Speakers
data/MFCC_Speaker_Clean_N_Train/train-clean-100/1841
data/Speakers/train-clean-100/1841/1841-159771-0006.flac
torch.Size([40, 1251])
3_000000_1841-159771-0006.pt
3_000001_1841-159771-0006.pt
Speakers_with_noise_clean
Speakers_with_noise
data/MFCC_Speaker_Noise_N_Train/train-clean-100/1841
data/Speakers_with_noise_clean/train-clean-100/1841/1841-159771-0006.flac
torch.Size([40, 1251])
3_000000_1841-159771-0006.pt
3_000001_1841-159771-0006.pt
Speakers
Speakers
data/MFCC_Speaker_Clean_N_Train/train-clean-100/1867
data/Speakers/train-clean-100/1867/1867-148436-0008.flac
torch.Size([40, 1251])
4_000000_1867-148436-0008.pt
4_000001_1867-148436-0008.pt
Speakers_with_noise_clean
Speakers_with_noise
data/MFCC_Speaker_Noise_N_Train/train-clean-100/1867
data/Speakers_with_noise_clean/train-clean-100/1867/1867-148436-0008.flac
torch.Size([40, 1251])
4_000000_1867-148436-0008.pt
4_000001_1867-148436-0008.pt
Speakers
Speakers
data/MFCC_Speaker_Clean_N_Train/train-clean-100/1963
dat

In [16]:
path_mfcc_folder_val_c_n = ['data/MFCC_Speaker_Clean_N_Val/', 'data/MFCC_Speaker_Noise_N_Val/']
LIST_MFCC_OF_ALL_SPEAKERS_VAL_C_N = generete_mfcc_pt(only_list_speakers_val_clean_noise, mfcc_speakers, path_mfcc_folder_val_c_n)

Speakers
Speakers
data/MFCC_Speaker_Clean_N_Val/train-clean-100/196
data/Speakers/train-clean-100/196/196-122152-0022.flac
torch.Size([40, 1251])
8_000000_196-122152-0022.pt
8_000001_196-122152-0022.pt
Speakers_with_noise_clean
Speakers_with_noise
data/MFCC_Speaker_Noise_N_Val/train-clean-100/196
data/Speakers_with_noise_clean/train-clean-100/196/196-122152-0022.flac
torch.Size([40, 1251])
8_000000_196-122152-0022.pt
8_000001_196-122152-0022.pt
Speakers
Speakers
data/MFCC_Speaker_Clean_N_Val/train-clean-100/1926
data/Speakers/train-clean-100/1926/1926-147979-0008.flac
torch.Size([40, 1251])
7_000000_1926-147979-0008.pt
7_000001_1926-147979-0008.pt
Speakers_with_noise_clean
Speakers_with_noise
data/MFCC_Speaker_Noise_N_Val/train-clean-100/1926
data/Speakers_with_noise_clean/train-clean-100/1926/1926-147979-0008.flac
torch.Size([40, 1251])
7_000000_1926-147979-0008.pt
7_000001_1926-147979-0008.pt
Speakers
Speakers
data/MFCC_Speaker_Clean_N_Val/train-clean-100/196
data/Speakers/train-clea

In [17]:
path_mfcc_folder_test_c_n = ['data/MFCC_Speaker_Clean_N_Test/', 'data/MFCC_Speaker_Noise_N_Test/']
LIST_MFCC_OF_ALL_SPEAKERS_TEST_C_N = generete_mfcc_pt(only_list_speakers_test_clean_noise, mfcc_speakers, path_mfcc_folder_test_c_n)

Speakers
Speakers
data/MFCC_Speaker_Clean_N_Test/train-clean-100/1963
data/Speakers/train-clean-100/1963/1963-142393-0002.flac
torch.Size([40, 1251])
9_000000_1963-142393-0002.pt
9_000001_1963-142393-0002.pt
Speakers_with_noise_clean
Speakers_with_noise
data/MFCC_Speaker_Noise_N_Test/train-clean-100/1963
data/Speakers_with_noise_clean/train-clean-100/1963/1963-142393-0002.flac
torch.Size([40, 1251])
9_000000_1963-142393-0002.pt
9_000001_1963-142393-0002.pt
Speakers
Speakers
data/MFCC_Speaker_Clean_N_Test/train-clean-100/19
data/Speakers/train-clean-100/19/19-227-0010.flac
torch.Size([40, 1251])
6_000000_19-227-0010.pt
6_000001_19-227-0010.pt
Speakers_with_noise_clean
Speakers_with_noise
data/MFCC_Speaker_Noise_N_Test/train-clean-100/19
data/Speakers_with_noise_clean/train-clean-100/19/19-227-0010.flac
torch.Size([40, 1251])
6_000000_19-227-0010.pt
6_000001_19-227-0010.pt
Speakers
Speakers
data/MFCC_Speaker_Clean_N_Test/train-clean-100/1841
data/Speakers/train-clean-100/1841/1841-150351

In [18]:
util.generate_pkl('variables/', LIST_MFCC_OF_ALL_SPEAKERS_TRAIN_C_N, 'LIST_MFCC_OF_ALL_SPEAKERS_TRAIN_C_N')
util.generate_pkl('variables/', LIST_MFCC_OF_ALL_SPEAKERS_VAL_C_N, 'LIST_MFCC_OF_ALL_SPEAKERS_VAL_C_N')
util.generate_pkl('variables/', LIST_MFCC_OF_ALL_SPEAKERS_TEST_C_N, 'LIST_MFCC_OF_ALL_SPEAKERS_TEST_C_N')

[[['data/MFCC_Speaker_Clean_N_Train/train-clean-100/1841/3_000000_1841-159771-0006.pt', 'data/MFCC_Speaker_Clean_N_Train/train-clean-100/1841/3_000001_1841-159771-0006.pt'], ['data/MFCC_Speaker_Noise_N_Train/train-clean-100/1841/3_000000_1841-159771-0006.pt', 'data/MFCC_Speaker_Noise_N_Train/train-clean-100/1841/3_000001_1841-159771-0006.pt']], [['data/MFCC_Speaker_Clean_N_Train/train-clean-100/1867/4_000000_1867-148436-0008.pt', 'data/MFCC_Speaker_Clean_N_Train/train-clean-100/1867/4_000001_1867-148436-0008.pt'], ['data/MFCC_Speaker_Noise_N_Train/train-clean-100/1867/4_000000_1867-148436-0008.pt', 'data/MFCC_Speaker_Noise_N_Train/train-clean-100/1867/4_000001_1867-148436-0008.pt']], [['data/MFCC_Speaker_Clean_N_Train/train-clean-100/1963/9_000000_1963-142393-0049.pt', 'data/MFCC_Speaker_Clean_N_Train/train-clean-100/1963/9_000001_1963-142393-0049.pt'], ['data/MFCC_Speaker_Noise_N_Train/train-clean-100/1963/9_000000_1963-142393-0049.pt', 'data/MFCC_Speaker_Noise_N_Train/train-clean-100

In [19]:
LIST_MFCC_OF_ALL_SPEAKERS_TRAIN_C_N

[[['data/MFCC_Speaker_Clean_N_Train/train-clean-100/1841/3_000000_1841-159771-0006.pt',
   'data/MFCC_Speaker_Clean_N_Train/train-clean-100/1841/3_000001_1841-159771-0006.pt'],
  ['data/MFCC_Speaker_Noise_N_Train/train-clean-100/1841/3_000000_1841-159771-0006.pt',
   'data/MFCC_Speaker_Noise_N_Train/train-clean-100/1841/3_000001_1841-159771-0006.pt']],
 [['data/MFCC_Speaker_Clean_N_Train/train-clean-100/1867/4_000000_1867-148436-0008.pt',
   'data/MFCC_Speaker_Clean_N_Train/train-clean-100/1867/4_000001_1867-148436-0008.pt'],
  ['data/MFCC_Speaker_Noise_N_Train/train-clean-100/1867/4_000000_1867-148436-0008.pt',
   'data/MFCC_Speaker_Noise_N_Train/train-clean-100/1867/4_000001_1867-148436-0008.pt']],
 [['data/MFCC_Speaker_Clean_N_Train/train-clean-100/1963/9_000000_1963-142393-0049.pt',
   'data/MFCC_Speaker_Clean_N_Train/train-clean-100/1963/9_000001_1963-142393-0049.pt'],
  ['data/MFCC_Speaker_Noise_N_Train/train-clean-100/1963/9_000000_1963-142393-0049.pt',
   'data/MFCC_Speaker_Noi

In [20]:
# only_list_speakers_train_dnn_noise
# only_list_speakers_val_dnn_noise
# only_list_speakers_test_dnn_noise

In [56]:
def generete_mfcc_pt(list_speakers, mfcc_tranforms, path_mfcc_folder):
    n_frame = 512
    count1 = 0
    count2 = 0
    num_samples_time = 16000*5
    LIST_MFCC_OF_ALL_SPEAKERS = []
    for speaker in list_speakers:
        MFCC_NOISE_SPEAKER_AUX = []
        MFCC_DENOISE_SPEAKER_AUX = []
        #print('1')
        #print(speaker[0])
        #print(speaker[1])
        #print('2')
        
        print(speaker[0][0])
        print(speaker[0][1])
        aux1 = speaker[0][1].split('/')
        print(aux1)
        audio, sample_rate = torchaudio.load(speaker[0][1])
        audio = cut_if_necessary(audio, num_samples_time)
        audio = right_pad_if_necessary(audio, num_samples_time)
        mfcc = mfcc_tranforms(audio).squeeze(0)
        print(mfcc.size())
        print(os.path.join(path_mfcc_folder[0], "/".join(aux1[0:len(aux1)-1])))
        Path(os.path.join(path_mfcc_folder[0], "/".join(aux1[0:len(aux1)-1]))).mkdir(parents=True, exist_ok=True)
        for j in np.arange(mfcc.shape[1] // n_frame):
            aux = mfcc.transpose(0,1)[j * n_frame:(j + 1) * n_frame]
            out_name =  str(speaker[0][0]) +'_'+ str(j).zfill(6) +'_'+ speaker[0][1].split('/')[3].split('.')[0] + '.pt'
            print(out_name)
            #print(os.path.join(path_mfcc_folder[0], "/".join(path_mfcc[1:len(path_mfcc)-1])))
            torch.save(aux, os.path.join(path_mfcc_folder[0], "/".join(aux1[0:len(aux1)-1]))+'/'+out_name)
            MFCC_DENOISE_SPEAKER_AUX.append(os.path.join(path_mfcc_folder[0], "/".join(aux1[0:len(aux1)-1]))+'/'+out_name)
            #count += 1
            
        print(speaker[1][0])
        print(speaker[1][1])
        aux2 = speaker[1][1].split('/')
        print(aux2)
        audio, sample_rate = torchaudio.load(speaker[1][1])
        audio = cut_if_necessary(audio, num_samples_time)
        audio = right_pad_if_necessary(audio, num_samples_time)
        mfcc = mfcc_tranforms(audio).squeeze(0)
        print(mfcc.size())
        print(os.path.join(path_mfcc_folder[1], "/".join(aux2[0:len(aux2)-1])))
        Path(os.path.join(path_mfcc_folder[1], "/".join(aux2[0:len(aux2)-1]))).mkdir(parents=True, exist_ok=True)
        for j in np.arange(mfcc.shape[1] // n_frame):
            aux = mfcc.transpose(0,1)[j * n_frame:(j + 1) * n_frame]
            out_name =  str(speaker[1][0]) +'_'+ str(j).zfill(6) +'_'+ speaker[1][1].split('/')[3].split('.')[0] + '.pt'
            print(out_name)
            #print(os.path.join(path_mfcc_folder[0], "/".join(path_mfcc[1:len(path_mfcc)-1])))
            torch.save(aux, os.path.join(path_mfcc_folder[1], "/".join(aux2[0:len(aux2)-1]))+'/'+out_name)
            MFCC_NOISE_SPEAKER_AUX.append(os.path.join(path_mfcc_folder[1], "/".join(aux2[0:len(aux2)-1]))+'/'+out_name)
            #count += 1
            
        #print(path_mfcc_folder[0])
        #print(path_mfcc_folder[1])
        #aux1 = speaker[0][1].split('/')
        #aux2 = speaker[1][1].split('/')
        #print(os.path.join(path_mfcc_folder[0], "/".join(aux1[0:len(aux1)-1])))
        #print(os.path.join(path_mfcc_folder[1], "/".join(aux2[0:len(aux2)-1])))
        
        
        
        
        #print('5')
        #print("/".join(aux1[0:len(aux1)-1]))
        #print("/".join(aux2[0:len(aux2)-1]))
        
        
        #print('4')
        #print("/".join(aux1[1:len(aux1)-1]))
        #print("/".join(aux2[2:len(aux2)-1]))
        #print('_______________________________')
        #print(path_mfcc_folder[0])
        #print(path_mfcc_folder[1])
        #print('_______________________________')
        
        #print('_______________________________')
        #print(os.path.join(path_mfcc_folder[0], "/".join(aux1[0:len(aux1)-1])))
        #print(os.path.join(path_mfcc_folder[1], "/".join(aux2[0:len(aux2)-1])))
        #print('_______________________________')
        
        
        LIST_MFCC_OF_ALL_SPEAKERS.append([MFCC_DENOISE_SPEAKER_AUX, MFCC_NOISE_SPEAKER_AUX])
        #print(count)
    return LIST_MFCC_OF_ALL_SPEAKERS 


In [57]:
path_mfcc_folder_train_d_n= ['data/MFCC_Speaker_Clean_D_Train/', 'data/MFCC_Speaker_Noise_D_Train/']
LIST_MFCC_OF_ALL_SPEAKERS_TRAIN_D_N = generete_mfcc_pt(only_list_speakers_train_dnn_noise, mfcc_speakers, path_mfcc_folder_train_d_n)

4
audios_denoised_dnn/9/stationary_rain/1867-154071-0031.flac
['audios_denoised_dnn', '9', 'stationary_rain', '1867-154071-0031.flac']
torch.Size([40, 1251])
data/MFCC_Speaker_Clean_D_Train/audios_denoised_dnn/9/stationary_rain
4_000000_1867-154071-0031.pt
4_000001_1867-154071-0031.pt
4
audios_denoised_dnn/Speakers_with_noise_dnn/stationary_rain/1867-154071-0031.flac
['audios_denoised_dnn', 'Speakers_with_noise_dnn', 'stationary_rain', '1867-154071-0031.flac']
torch.Size([40, 1251])
data/MFCC_Speaker_Noise_D_Train/audios_denoised_dnn/Speakers_with_noise_dnn/stationary_rain
4_000000_1867-154071-0031.pt
4_000001_1867-154071-0031.pt
7
audios_denoised_dnn/6/stationary_car/1926-147979-0013.flac
['audios_denoised_dnn', '6', 'stationary_car', '1926-147979-0013.flac']
torch.Size([40, 1251])
data/MFCC_Speaker_Clean_D_Train/audios_denoised_dnn/6/stationary_car
7_000000_1926-147979-0013.pt
7_000001_1926-147979-0013.pt
7
audios_denoised_dnn/Speakers_with_noise_dnn/stationary_car/1926-147979-0013.f

In [58]:
path_mfcc_folder_val_d_n = ['data/MFCC_Speaker_Clean_D_Val/', 'data/MFCC_Speaker_Noise_D_Val']
LIST_MFCC_OF_ALL_SPEAKERS_VAL_D_N = generete_mfcc_pt(only_list_speakers_val_dnn_noise, mfcc_speakers, path_mfcc_folder_val_d_n)

8
audios_denoised_dnn/-9/nonstationary_people/196-122152-0015.flac
['audios_denoised_dnn', '-9', 'nonstationary_people', '196-122152-0015.flac']
torch.Size([40, 1251])
data/MFCC_Speaker_Clean_D_Val/audios_denoised_dnn/-9/nonstationary_people
8_000000_196-122152-0015.pt
8_000001_196-122152-0015.pt
8
audios_denoised_dnn/Speakers_with_noise_dnn/nonstationary_people/196-122152-0015.flac
['audios_denoised_dnn', 'Speakers_with_noise_dnn', 'nonstationary_people', '196-122152-0015.flac']
torch.Size([40, 1251])
data/MFCC_Speaker_Noise_D_Val/audios_denoised_dnn/Speakers_with_noise_dnn/nonstationary_people
8_000000_196-122152-0015.pt
8_000001_196-122152-0015.pt
0
audios_denoised_dnn/0/nonstationary_bell_church/1723-141149-0068.flac
['audios_denoised_dnn', '0', 'nonstationary_bell_church', '1723-141149-0068.flac']
torch.Size([40, 1251])
data/MFCC_Speaker_Clean_D_Val/audios_denoised_dnn/0/nonstationary_bell_church
0_000000_1723-141149-0068.pt
0_000001_1723-141149-0068.pt
0
audios_denoised_dnn/Speak

In [59]:
path_mfcc_folder_test_d_n = ['data/MFCC_Speaker_Clean_D_Test/', 'data/MFCC_Speaker_Noise_D_Test/']
LIST_MFCC_OF_ALL_SPEAKERS_TEST_D_N = generete_mfcc_pt(only_list_speakers_test_dnn_noise, mfcc_speakers, path_mfcc_folder_test_d_n)

3
audios_denoised_dnn/9/stationary_wind/1841-159771-0012.flac
['audios_denoised_dnn', '9', 'stationary_wind', '1841-159771-0012.flac']
torch.Size([40, 1251])
data/MFCC_Speaker_Clean_D_Test/audios_denoised_dnn/9/stationary_wind
3_000000_1841-159771-0012.pt
3_000001_1841-159771-0012.pt
3
audios_denoised_dnn/Speakers_with_noise_dnn/stationary_wind/1841-159771-0012.flac
['audios_denoised_dnn', 'Speakers_with_noise_dnn', 'stationary_wind', '1841-159771-0012.flac']
torch.Size([40, 1251])
data/MFCC_Speaker_Noise_D_Test/audios_denoised_dnn/Speakers_with_noise_dnn/stationary_wind
3_000000_1841-159771-0012.pt
3_000001_1841-159771-0012.pt
6
audios_denoised_dnn/-9/stationary_wind/19-227-0026.flac
['audios_denoised_dnn', '-9', 'stationary_wind', '19-227-0026.flac']
torch.Size([40, 1251])
data/MFCC_Speaker_Clean_D_Test/audios_denoised_dnn/-9/stationary_wind
6_000000_19-227-0026.pt
6_000001_19-227-0026.pt
6
audios_denoised_dnn/Speakers_with_noise_dnn/stationary_wind/19-227-0026.flac
['audios_denoised

In [60]:
util.generate_pkl('variables/', LIST_MFCC_OF_ALL_SPEAKERS_TRAIN_D_N, 'LIST_MFCC_OF_ALL_SPEAKERS_TRAIN_D_N')
util.generate_pkl('variables/', LIST_MFCC_OF_ALL_SPEAKERS_VAL_D_N, 'LIST_MFCC_OF_ALL_SPEAKERS_VAL_D_N')
util.generate_pkl('variables/', LIST_MFCC_OF_ALL_SPEAKERS_TEST_D_N, 'LIST_MFCC_OF_ALL_SPEAKERS_TEST_D_N')

[[['data/MFCC_Speaker_Clean_D_Train/audios_denoised_dnn/9/stationary_rain/4_000000_1867-154071-0031.pt', 'data/MFCC_Speaker_Clean_D_Train/audios_denoised_dnn/9/stationary_rain/4_000001_1867-154071-0031.pt'], ['data/MFCC_Speaker_Noise_D_Train/audios_denoised_dnn/Speakers_with_noise_dnn/stationary_rain/4_000000_1867-154071-0031.pt', 'data/MFCC_Speaker_Noise_D_Train/audios_denoised_dnn/Speakers_with_noise_dnn/stationary_rain/4_000001_1867-154071-0031.pt']], [['data/MFCC_Speaker_Clean_D_Train/audios_denoised_dnn/6/stationary_car/7_000000_1926-147979-0013.pt', 'data/MFCC_Speaker_Clean_D_Train/audios_denoised_dnn/6/stationary_car/7_000001_1926-147979-0013.pt'], ['data/MFCC_Speaker_Noise_D_Train/audios_denoised_dnn/Speakers_with_noise_dnn/stationary_car/7_000000_1926-147979-0013.pt', 'data/MFCC_Speaker_Noise_D_Train/audios_denoised_dnn/Speakers_with_noise_dnn/stationary_car/7_000001_1926-147979-0013.pt']], [['data/MFCC_Speaker_Clean_D_Train/audios_denoised_dnn/6/stationary_cabin/5_000000_1898

In [61]:
LIST_MFCC_OF_ALL_SPEAKERS_TRAIN_D_N

[[['data/MFCC_Speaker_Clean_D_Train/audios_denoised_dnn/9/stationary_rain/4_000000_1867-154071-0031.pt',
   'data/MFCC_Speaker_Clean_D_Train/audios_denoised_dnn/9/stationary_rain/4_000001_1867-154071-0031.pt'],
  ['data/MFCC_Speaker_Noise_D_Train/audios_denoised_dnn/Speakers_with_noise_dnn/stationary_rain/4_000000_1867-154071-0031.pt',
   'data/MFCC_Speaker_Noise_D_Train/audios_denoised_dnn/Speakers_with_noise_dnn/stationary_rain/4_000001_1867-154071-0031.pt']],
 [['data/MFCC_Speaker_Clean_D_Train/audios_denoised_dnn/6/stationary_car/7_000000_1926-147979-0013.pt',
   'data/MFCC_Speaker_Clean_D_Train/audios_denoised_dnn/6/stationary_car/7_000001_1926-147979-0013.pt'],
  ['data/MFCC_Speaker_Noise_D_Train/audios_denoised_dnn/Speakers_with_noise_dnn/stationary_car/7_000000_1926-147979-0013.pt',
   'data/MFCC_Speaker_Noise_D_Train/audios_denoised_dnn/Speakers_with_noise_dnn/stationary_car/7_000001_1926-147979-0013.pt']],
 [['data/MFCC_Speaker_Clean_D_Train/audios_denoised_dnn/6/stationary_c